# Fork Long Training Runs with Neptune

<a target="_blank" href="https://colab.research.google.com/github/neptune-ai/scale-examples/blob/lb/forking-long-runs/how-to-guides/debug-model-training-runs/debug_training_runs.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/> 
</a>
<a target="_blank" href="https://github.com/neptune-ai/scale-examples/blob/lb/forking-long-runs/how-to-guides/debug-model-training-runs/debug_training_runs.ipynb">
  <img alt="Open in GitHub" src="https://img.shields.io/badge/Open_in_GitHub-blue?logo=github&labelColor=black">
</a>
<a target="_blank" href="https://docs-beta.neptune.ai/tutorials/">
  <img alt="View tutorial in docs" src="https://neptune.ai/wp-content/uploads/2024/01/docs-badge-2.svg">
</a>

## Introduction
Training large models often requires running experiments for weeks or even months. During these extended training periods, you may need to:
- Fork your training runs to explore different hyperparameters or architectures
- Resume training from checkpoints after interruptions
- Monitor long-term training stability and convergence

Neptune provides robust support for managing long-running training experiments, allowing you to:
- Fork existing runs to explore variations without losing progress
- Track metrics and artifacts across multiple months of training
- Maintain experiment lineage and reproducibility

In this tutorial, you'll learn how to:
1. **Initialize Neptune** and **log configuration parameters**
2. **Fork existing training runs** to explore variations
3. **Resume training** from checkpoints
4. **Monitor long-term training stability**

Step through a pre-configured report:
<a target="_blank" href="https://scale.neptune.ai/leo/pytorch-tutorial/reports/9e79d952-272a-4a38-83e5-27df4dd225ec">
  <img alt="Explore in Neptune" src="https://neptune.ai/wp-content/uploads/2024/01/neptune-badge.svg">
</a>

_Note: This is a code recipe that you can adapt for your own model training needs._

## Before you start

  1. Create a Neptune Scale account. [Register &rarr;](https://neptune.ai/early-access)
  2. Create a Neptune project for tracking metadata. For instructions, see [Projects](https://docs-beta.neptune.ai/projects/) in the Neptune Scale docs.
  3. Install and configure Neptune Scale for logging metadata. For instructions, see [Get started](https://docs-beta.neptune.ai/setup) in the Neptune Scale docs.

### Set environment variables
Set your project name and API token as environment variables to log to your Neptune Scale project.

Uncomment the code block below and replace placeholder values with your own credentials:

In [ ]:
# Set Neptune credentials as environment variables
# %env NEPTUNE_API_TOKEN = YOUR_API_TOKEN
# %env NEPTUNE_PROJECT = WORKSPACE_NAME/PROJECT_NAME

### Install dependencies and import libraries

In [ ]:
# Install dependencies
! pip install -qU neptune_scale torch datasets

import torch

### Initialize parameters

In [94]:
params = {
    "optimizer": "Adam",
    "batch_size": 512,
    "learning_rate": 0.01,
    "epochs": 3,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "input_features": 256,
    "embed_size": 1000,
    "hidden_size": 256,  # hidden size for the LSTM
    "dropout_prob": 0.3,
    "num_lstm_layers": 3,
}

### Setup data, model and other dependencies

In the next cell, we'll:
1. Import required PyTorch and HuggingFace libraries
2. Download and load a [next token prediction dataset](https://huggingface.co/datasets/Na0s/Next_Token_Prediction_dataset) from HuggingFace
3. Create PyTorch DataLoaders for training
4. Calculate vocabulary size for the model
5. Define a multilayer model for training

_You can modify this setup to use your own data and model architecture._

In [98]:
## Before you start
from torch import nn
import torch.optim as optim
from datasets import load_dataset
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F

def setup_dataset(params: dict, data_files):
    # Download dataset
    base_url = "https://huggingface.co/datasets/Na0s/Next_Token_Prediction_dataset/resolve/main/data/"

    if isinstance(data_files, str):
        data_files = [data_files]

    data_files = {
        "train": [base_url + file for file in data_files]
    }

    # Load dataset
    data_subset = load_dataset("parquet", data_files=data_files, num_proc=4)
    # validation_subset = data_subset.get("validation").train_test_split(test_size=0.1)

    print(f"Training samples: {data_subset['train'].num_rows}")
    # print(f"Validation samples: {validation_subset['test'].num_rows}")

    # Convert to PyTorch format
    train_subset = data_subset["train"].with_format(
        type="torch", columns=["text", "input_ids", "labels"]
    )
    """validation_subset = validation_subset["test"].with_format(
        type="torch", columns=["text", "input_ids", "labels"]
    )"""

    # Create dataloaders
    train_dataloader = DataLoader(train_subset, batch_size=params["batch_size"], shuffle=True)
    # val_dataloader = DataLoader(validation_subset, batch_size=params["batch_size"], shuffle=True)

    # Calculate vocabulary size
    params["vocab_size"] = (
        max([token for sentence in data_subset["train"]["input_ids"] for token in sentence]) + 1
    )
    print(f"Vocabulary size: {params['vocab_size']}")

    return train_dataloader

def create_mnist_dataloader(params):
    """
    Create a DataLoader for the MNIST dataset.
    
    Args:
        batch_size (int): Number of samples per batch
        num_workers (int): Number of subprocesses to use for data loading
        shuffle (bool): Whether to shuffle the data
    
    Returns:
        DataLoader: PyTorch DataLoader for MNIST training data
    """
    # Define transformations
    transform = transforms.Compose([
        transforms.ToTensor(),  # Convert to tensor
        transforms.Normalize((0.1307,), (0.3081,))  # MNIST mean and std
    ])
    
    # Download and load training data
    train_dataset = datasets.MNIST(
        root='./data',  # Directory to save the data
        train=True,     # Use training set
        download=True,  # Download if not already present
        transform=transform
    )
    
    # Create DataLoader
    train_loader = DataLoader(
        train_dataset,
        batch_size=params["batch_size"],
        shuffle=True,
    )
    
    return train_loader

# Working with PyTorch checkpoints
def save_model_state(model, optimizer, step_counter, epoch):
    
    save_path=f"model_state_step_{step_counter}_epoch_{epoch}.pt"
    
    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'step': step_counter
    }, save_path)

def load_model_state(model, optimizer, step_counter, epoch):

    load_path=f"model_state_step_{step_counter}_epoch_{epoch}.pt"
    checkpoint = torch.load(load_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return checkpoint

class SimpleMNISTModel(nn.Module):
    def __init__(self):
        super(SimpleMNISTModel, self).__init__()
        
        # Convolutional layers
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        # Max pooling
        self.pool = nn.MaxPool2d(2, 2)
        
        # Fully connected layers
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # 7*7 comes from image size after pooling
        self.fc2 = nn.Linear(128, 10)  # 10 classes for MNIST
        
        # Dropout
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        # First conv block
        x = self.pool(F.relu(self.conv1(x)))
        
        # Second conv block
        x = self.pool(F.relu(self.conv2(x)))
        
        # Flatten
        x = x.view(-1, 64 * 7 * 7)
        
        # Fully connected layers
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

# Create model
class MultilayerModel(nn.Module):
    """A larger language model with multiple LSTM and fully connected layers."""

    def __init__(self, vocab_size, embed_size, hidden_size):
        super(MultilayerModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)

        # Create multiple LSTM layers
        self.lstm_layers = nn.ModuleList(
            [
                nn.LSTM(
                    hidden_size if i > 0 else embed_size,
                    hidden_size,
                    num_layers=1,
                    batch_first=True,
                )
                for i in range(10)  # 10 LSTM layers
            ]
        )

        # Create multiple fully connected layers
        self.fc_layers = nn.ModuleList(
            [nn.Linear(hidden_size, hidden_size) for _ in range(9)]  # 9 FC layers
        )

        # Final layer to project back to vocab size
        self.final_layer = nn.Linear(hidden_size, vocab_size)

        # Add dropout for regularization
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        # Embedding layer
        x = self.embedding(x)

        # Process through LSTM layers
        for lstm in self.lstm_layers:
            x, _ = lstm(x)
            x = self.dropout(x)

        # Process through FC layers
        for fc in self.fc_layers:
            x = fc(x)
            x = self.dropout(x)
            x = torch.relu(x)

        # Final projection
        out = self.final_layer(x)
        return out

dataset_file = "train-00001-of-00067.parquet"
train_dataloader = create_mnist_dataloader(params)
'''model = MultilayerModel(
    params["vocab_size"],
    params["embed_size"],
    params["hidden_size"],
)'''
model = SimpleMNISTModel()
model.to(params["device"])
print(f"Model created: {model.__class__.__name__}")
optimizer = optim.Adam(model.parameters(), lr=params["learning_rate"])
print(f"Optimizer: {optimizer.__class__.__name__}")
criterion = nn.CrossEntropyLoss(
    ignore_index=-100
)  # Ignore the buffering index of -100 in the dataset
print(f"Criterion: {criterion.__class__.__name__}")

Model created: SimpleMNISTModel
Optimizer: Adam
Criterion: CrossEntropyLoss


## Debug model training run with Neptune

### Step 1: _Initialize Neptune Run object_

The `Run` object is used to log configuration parameters and metrics. 

In [99]:
from neptune_scale import Run

run = Run(
    # run_id = "base-run", # Example of a custom run_id and illustration of this example
    experiment_name="forking-text",  # Create a run that is the head of an experiment.
)

print(run.get_experiment_url())

https://scale.neptune.ai/leo/pytorch-tutorial/runs/details?runIdentificationKey=forking-text&type=experiment


### Step 2: _Log configuration parameters and tags_

In [100]:
run.log_configs(
    {
        "config/learning_rate": params["learning_rate"],
        "config/optimizer": params["optimizer"],
        "config/batch_size": params["batch_size"],
        "config/epochs": params["epochs"],
        "config/num_lstm_layers": params["num_lstm_layers"],
        "data/embed_size": params["embed_size"],
        "sys/description": "Base experiment used to get an inital version of the model"
    }
)

run.add_tags(tags=["text", "forking", params["optimizer"]])
run.add_tags([dataset_file], group_tags=True)

print(f"See configuration parameters:\n{run.get_experiment_url() + '&detailsTab=metadata'}")

See configuration parameters:
https://scale.neptune.ai/leo/pytorch-tutorial/runs/details?runIdentificationKey=forking-text&type=experiment&detailsTab=metadata


### Step 3: _Execute model training loop_

In this training loop, we:
1. Register backward hooks to capture gradient norms from all model layers with `register_full_backward_hook()`
2. Track these norms during training to identify potential issues like vanishing/exploding gradients in a dictionary called `debugging_gradient_norms`
3. Log the gradient norms to Neptune for visualization and analysis using the `log_metrics` method

This approach allows you to monitor the learning dynamics across your entire model architecture in near real-time.

In [101]:
'''
# Register hooks to track gradients for each layer
        "config/learning_rate": params["learning_rate"],
        "config/optimizer": params["optimizer"],
        "config/batch_size": params["batch_size"],
        "config/epochs": params["epochs"],
        "config/num_lstm_layers": params["num_lstm_layers"],
        "data/embed_size": params["embed_size"],

# Define dictionary of metrics to log to Neptune
debugging_gradient_norms = {}
# Register hooks once before training
for name, module in model.named_modules():
    module.register_full_backward_hook(hook_fn)
    '''

# Create custom Neptune URLS for tutorial steps
print(f"View charts in near real-time:\n{run.get_experiment_url() + '&detailsTab=charts'}")

step_counter = 0
# Training loop
for epoch in range(params["epochs"]):
    total_loss = 0
    for batch in train_dataloader:
        model.train()
        step_counter += 1
        # input_ids = batch[0].to(params["device"])
        images = batch[0].to(params["device"])
        labels = batch[1].to(params["device"])
        optimizer.zero_grad()
        # logits = model(input_ids)
        # loss = criterion(logits.view(-1, params["vocab_size"]), labels.view(-1))
        outputs = model(images)  # No need for .view() with MNIST
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Log global training loss and layer-wise gradient norms
        run.log_metrics(
            data={"metrics/train/loss": loss.item()},
            step=step_counter,
        )

    print(f"Saving model checkpoint epoch: {epoch}")
    save_model_state(model, optimizer, step_counter, epoch)


View charts in near real-time:
https://scale.neptune.ai/leo/pytorch-tutorial/runs/details?runIdentificationKey=forking-text&type=experiment&detailsTab=charts
Saving model checkpoint epoch: 0
Saving model checkpoint epoch: 1
Saving model checkpoint epoch: 2


In [74]:

# Close run to ensure all operations are processed
run.close()

2025-04-14 17:57:02,317 neptune:INFO: Waiting for all operations to be processed
2025-04-14 17:57:02,320 neptune:WARNING: No timeout specified. Waiting indefinitely
2025-04-14 17:57:02,321 neptune:INFO: All operations were processed


In [ ]:
# Need to update parameters, model architecture or dataset

params = {
    "batch_size": 8,
    "learning_rate": 0.01,
    "epochs": 5,
    "input_features": 256,
    "dropout_prob": 0.3,
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
}

# Add additional dataset
dataset_file = ["train-00001-of-00067.parquet", "train-00002-of-00067.parquet"]
train_dataloader = setup_dataset(params, dataset_file)

'''
model = MultilayerModel(
    params["vocab_size"],
    params["embed_size"],
    params["hidden_size"],
)'''

optimizer = optim.Adam(model.parameters(), lr=params["learning_rate"])
# Load model state from saved checkpoint
load_model_state(model, optimizer, step_counter=100, epoch=0)

# Use a learning rate scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="min",
    factor=0.1,
    patience=3.5,
    verbose=True
)

model.to(params["device"])
print(f"Model created: {model.__class__.__name__}")
optimizer = optim.Adam(model.parameters(), lr=params["learning_rate"])
print(f"Optimizer: {optimizer.__class__.__name__}")
criterion = nn.CrossEntropyLoss(
    ignore_index=-100
)  # Ignore the buffering index of -100 in the dataset
print(f"Criterion: {criterion.__class__.__name__}")

Training samples: 163852
Vocabulary size: 128257
Model created: MultilayerModel
Optimizer: Adam
Criterion: CrossEntropyLoss


In [79]:
# Create a forked run
print(run._run_id)
run = Run(
    #experiment_name="forking-text",
    # run_id="forked-run",
    fork_run_id = "creative-batch-20250414154819921-69jt8",
    fork_step = 100
)

run.log_configs(
    {
        "config/learning_rate": params["learning_rate"],
        "config/batch_size": params["batch_size"],
        "config/epochs": params["epochs"],
    }
)

# run.add_tags(tags=["text", "forking", params["optimizer"]])
run.add_tags(dataset_file, group_tags=True)

print(run.get_run_url())

fortified-frequency-20250414160234875-75smj
https://scale.neptune.ai/leo/pytorch-tutorial/-/run/?customId=industrial-hypothesis-20250414160311411-wv20s


In [80]:
step_counter = 101
# Training loop
for epoch in range(params["epochs"]):
    total_loss = 0
    for batch in train_dataloader:
        model.train()
        step_counter += 1
        input_ids = batch["input_ids"].to(params["device"])
        labels = batch["labels"].to(params["device"])
        optimizer.zero_grad()
        logits = model(input_ids)
        loss = criterion(logits.view(-1, params["vocab_size"]), labels.view(-1))
        loss.backward()
        optimizer.step()

        # Update learning rate based on step
        scheduler.step(loss)

        # Log global training loss and layer-wise gradient norms
        run.log_metrics(
            data={"metrics/train/loss": loss.item()},
            step=step_counter,
        )

KeyboardInterrupt: 

In [81]:
run.close()

2025-04-14 18:08:09,170 neptune:INFO: Waiting for all operations to be processed
2025-04-14 18:08:09,172 neptune:WARNING: No timeout specified. Waiting indefinitely
2025-04-14 18:08:09,173 neptune:INFO: All operations were processed


### Step 4: _Analyze and debug training_
While the model is training, you can start using the Neptune web app to browse your metrics and create custom analyses and visualizations:
1. To visualize the large number of metrics being logged in near real time, navigate to the **Charts** tab of the active run (_or select link above_).
2. Filter the metrics using the [advanced regex searching capabilities](https://docs-beta.neptune.ai/charts#filtering-charts). For example, enter `gradient & fc & layers.[0-5] & norm` in the search bar. This query filters the metrics for the first 6 layers of the gradients norms of the fully connected layers. You can specify down to exactly the metrics name you want.

![Alt text](tutorial-images/debugging_regex_search.png)

3. Export the filter to a [dashboard](https://docs-beta.neptune.ai/custom_dashboard). The saved dashboard will now only display these metrics during training. This is useful if you know that a certain set of layers can be troublesome during training. 
4. Alternatively, use the [dynamic metric selection](https://docs-beta.neptune.ai/chart_widget#dynamic-metric-selection) and create a new chart widget to display all LSTM layers gradient norms in one chart. Again, use the `(.*gradient)(.*lstm)(.*norm)` query. This makes it easy to have an automatically updating chart that allows you to view all layers on a single chart for rapid debugging in case vanishing or exploding gradients appear. 

![Alt text](tutorial-images/debugging_dashboard.png)

5. To document this behavior, create a [custom report](https://docs-beta.neptune.ai/reports) to outline the model training, global metrics, debugging metrics for the model you're training. This allows you to keep track of any anomalies but also to see what worked or did not work during training.

![Alt text](tutorial-images/debugging_report.png)

See the pre-configured [example of the training report](https://scale.neptune.ai/leo/pytorch-tutorial/reports/9e79d952-272a-4a38-83e5-27df4dd225ec).

See also: PyTorch layer-wise tracking package [here](TODO:Link to integration for tracking layer-wise metrics)
